In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv(r"C:\Users\lstreitm\Documents\Sienna\ITL Analysis NAERM\EIA BA Limits v2 - Updated\EIA_BAlims_2021-2023v2_full.csv")

from_var = "fromferc"
to_var = "toferc"

data = data.loc[data[from_var] != data[to_var]] # remove intra-regional transfers
#data.drop(columns=["fromferc","toferc"],inplace=True)
data_exp = data.loc[data["value"] >= 0].copy()
data_exp["Reporter"] = data_exp[from_var]
data_imp = data.loc[data["value"] <= 0].copy()
data_imp["Reporter"] = data_imp[from_var]
data_imp.rename(columns={from_var:to_var,to_var:from_var},inplace=True)
data_imp["value"] *= -1

df = pd.concat([data_imp, data_exp], axis=0)
df["Combined"] = df[[from_var, to_var]].apply(tuple, 1).apply(sorted).apply(tuple)

In [3]:
df

,period,toferc,fromferc,value,Reporter,Combined
1,2021-01-01T00,CAISO,Mexico,31,CAISO,"(CAISO, Mexico)"
2,2021-01-01T00,CAISO,NorthernGrid,3802,CAISO,"(CAISO, NorthernGrid)"
3,2021-01-01T00,CAISO,WestConnect,3382,CAISO,"(CAISO, WestConnect)"
4,2021-01-01T00,ERCOT,Mexico,77,ERCOT,"(ERCOT, Mexico)"
5,2021-01-01T00,ERCOT,SPP,394,ERCOT,"(ERCOT, SPP)"
...,...,...,...,...,...,...
1223454,2024-01-01T00,SERC,SPP,537,SPP,"(SERC, SPP)"
1223456,2024-01-01T00,Saskatchewan,SPP,8,SPP,"(SPP, Saskatchewan)"
1223457,2024-01-01T00,WestConnect,SPP,386,SPP,"(SPP, WestConnect)"
1223458,2024-01-01T00,CAISO,WestConnect,4488,WestConnect,"(CAISO, WestConnect)"


## Flow duration curves

In [90]:
interfaces = list(set(df["Combined"]))
percentile = 0.999

for (one,two) in interfaces:
    forward = df.loc[(df[from_var] == one) & (df[to_var] == two)]
    reverse = df.loc[(df[from_var] == two) & (df[to_var] == one)]
    try:
        forward_lim = forward.groupby([from_var, to_var])["value"].quantile(percentile, interpolation="lower").values[0]
    except:
        forward_lim = 1000000
    try:
        reverse_lim = reverse.groupby([from_var, to_var])["value"].quantile(percentile, interpolation="lower").values[0]
    except:
        reverse_lim = 1000000
    forward = forward[forward["value"]<forward_lim]
    reverse = reverse[reverse["value"]<reverse_lim]
    ax = plt.axes()
    ax.plot(forward[forward["Reporter"]==one]["value"].sort_values(ascending=False).reset_index(drop=True), label=f"Forward (Reported by {one})",color="red")
    ax.plot(forward[forward["Reporter"]==two]["value"].sort_values(ascending=False).reset_index(drop=True), label=f"Forward (Reported by {two})",color="blue")
    ax.plot((reverse[reverse["Reporter"]==one]["value"].sort_values(ascending=False)*-1).reset_index(drop=True), label=f"Reverse (Reported by {one})",color="red",linestyle="dashed")
    ax.plot((reverse[reverse["Reporter"]==two]["value"].sort_values(ascending=False)*-1).reset_index(drop=True), label=f"Reverse (Reported by {two})",color="blue",linestyle="dashed")
    ax.set_xlim(left=0)
    ax.set_title(f"{one}-{two} Historical Flows (2021-2023)")
    ax.set_ylabel("MW Transfer")
    ax.set_xlabel("Number of Hours")
    ax.legend()
    plt.savefig(f"{one}-{two}_HistoricalDurationCurve.png")
    plt.close()
    

## Extract Maxes

In [32]:
percentile = 0.99
df_max = df.groupby(["fromferc", "toferc","Reporter"])["value"].max().to_frame("Max")
df_max_avg = df_max.groupby(["fromferc", "toferc"])["Max"].mean().to_frame("Max_Avg")
df_max_max = df_max.groupby(["fromferc", "toferc"])["Max"].max().to_frame("Max_Max")

df_999 = df.groupby(["fromferc", "toferc","Reporter"])["value"].quantile(percentile, interpolation="lower").to_frame(f"Q{percentile}")
df_999_avg = df_999.groupby(["fromferc", "toferc"])[f"Q{percentile}"].mean().to_frame(f"Q{percentile}_Avg")
df_999_max = df_999.groupby(["fromferc", "toferc"])[f"Q{percentile}"].max().to_frame(f"Q{percentile}_Max")

df_hist = pd.concat([df_max_avg,df_max_max,df_999_avg,df_999_max], axis=1)#.reset_index()
#df_hist.to_csv("historicals_2021-2023.csv")

In [33]:
df_999[(df_999.index.get_level_values(0)== "PJM")&(df_999.index.get_level_values(1)== "NYISO")]

Q0.99
fromferc toferc Reporter       
PJM      NYISO  NYISO      3540
                PJM        4383